In [3]:
import cPickle
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
import operator
import pandas.io.data
from sklearn.qda import QDA
import re
from dateutil import parser
#from backtest import Strategy, Portfolio

In [7]:
symbol = 'GOOG'
df =  pd.io.data.get_data_yahoo(symbol)
df.columns.values[-1] = 'AdjClose'
df.columns = df.columns + '_' + symbol
df['Return_%s' %symbol] = df['AdjClose_%s' %symbol].pct_change()

In [9]:
def addFeatures(dataframe, adjclose, returns, n):
    """
    operates on two columns of dataframe:
    - n >= 2
    - given Return_* computes the return of day i respect to day i-n. 
    - given AdjClose_* computes its moving average on n days
 
    """
    
    return_n = adjclose[9:] + "Time" + str(n)
    dataframe[return_n] = dataframe[adjclose].pct_change(n)
    
    roll_n = returns[7:] + "RolMean" + str(n)
    dataframe[roll_n] = pd.rolling_mean(dataframe[returns], n)

In [11]:
delta = [2,3,4,5]
columns = df.columns    
adjclose = columns[-2]
returns = columns[-1]
for n in delta:
    addFeatures(df, adjclose, returns, n)

/home/manik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=2,center=False).mean()
/home/manik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=3,center=False).mean()
/home/manik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=4,center=False).mean()
/home/manik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:14: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()


In [13]:
df.Return_GOOG = df.Return_GOOG.shift(-1)

In [22]:
df = df.iloc[:, 6:]

In [24]:
columns = df.columns[::(2*max(delta)-1)]
for col in columns:
    print col

Return_GOOG


In [28]:
#just selecting the non NaN values of the data frame
df2 = df.iloc[5:-1,:]


,Return_GOOG,GOOGTime2,GOOGRolMean2,GOOGTime3,GOOGRolMean3,GOOGTime4,GOOGRolMean4,GOOGTime5,GOOGRolMean5
Date,,,,,,,,,
2010-01-11,-0.017684,0.011799,0.005910,-0.011755,-0.003820,-0.036667,-0.009167,-0.040909,-0.008214
2010-01-12,-0.005741,-0.019169,-0.009598,-0.006093,-0.001955,-0.029231,-0.007286,-0.053703,-0.010871
2010-01-13,0.004701,-0.023324,-0.011713,-0.024800,-0.008312,-0.011799,-0.002901,-0.034804,-0.006977
2010-01-14,-0.016699,-0.001067,-0.000520,-0.018732,-0.006241,-0.020215,-0.005059,-0.007154,-0.001381
2010-01-15,0.013138,-0.012077,-0.005999,-0.017748,-0.005913,-0.035118,-0.008856,-0.036577,-0.007387
2010-01-19,-0.012270,-0.003781,-0.001781,0.000903,0.000380,-0.004844,-0.001150,-0.022442,-0.004457
2010-01-20,0.004428,0.000707,0.000434,-0.016004,-0.005277,-0.011378,-0.002782,-0.017054,-0.003374
2010-01-21,-0.056554,-0.007896,-0.003921,0.005138,0.001765,-0.011647,-0.002851,-0.007001,-0.001340
2010-01-22,-0.018200,-0.052377,-0.026063,-0.064004,-0.021465,-0.051707,-0.012815,-0.067543,-0.013591


In [36]:
le = preprocessing.LabelEncoder()
df2['UpDown'] = df2['Return_GOOG']
df2.UpDown[df.Return_GOOG >= 0] = 'Up'
df2.UpDown[df.Return_GOOG < 0] = 'Down'

/home/manik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/manik/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [38]:
df2.UpDown = le.fit(df2.UpDown).transform(df2.UpDown)

/home/manik/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2698: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [40]:
features = df2.columns[1:-1]
X = df2[features]    
y = df2.UpDown

In [45]:
X.shape

(1803, 8)

In [52]:
start_test = 1200
X_train = X.iloc[0:start_test,:]
y_train = y.iloc[0:start_test]              
    
X_test = X.iloc[start_test:-1,:]    
y_test = y.iloc[start_test:-1]

In [54]:
print X_train.shape
print X_test.shape
print y_train.shape
print y_test.shape

(1200, 8)
(602, 8)
(1200,)
(602,)


In [55]:
clf = RandomForestClassifier(n_estimators=1000, n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [56]:
accuracy = clf.score(X_test, y_test)

In [57]:
print accuracy

0.513289036545


In [58]:
clf = neighbors.KNeighborsClassifier()
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)

In [59]:
print accuracy

0.511627906977
